In [ ]:
#!jupyter nbconvert --to=python CO2fluxSpatialTakahashi_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class CO2f_Takahashi_comp:
    '''
    class CO2f_Takahashi_comp(runname,resultpath,savepath,meshpath,txtfile,first_year,last_year,
                 mapproj='pc',savefig=False,layerwise=False)
    '''
    def __init__(self,runname,resultpath,savepath,meshpath,txtfile,first_year,last_year,
                 mapproj='pc',
                 savefig=False,
                 layerwise=False,depth_array=[]):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.meshpath = meshpath
        self.txtfile = txtfile
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.savefig = savefig
        self.layerwise = layerwise
        self.depth_array = depth_array

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        from scipy.interpolate import griddata
        import skill_metrics as sm
        import cartopy.crs as ccrs
        #import pickle

        import pyfesom2 as pf

        from plot_Taylor_normalized import plt_Taylor_norm

        # load FESOM mesh -------------------------------------------------------------------------------------
        mesh       = pf.load_mesh(meshpath)
        years = [self.fyear, self.lyear]


        # load FESOM data -------------------------------------------------------------------------------------
        # check variables
        #NCfesom = self.resultpath + '/CO2f.'+self.runname+'.'+str(self.fyear)+'.nc'
        #!ncdump -h $NCfesom
        
        # CO2f:description = "CO2-flux into the surface water" ;
        # CO2f:units = "mmolC/m2/d"
        
        # var_id ='CO2f'         # CO2 flux mmol/m2/d
        CO2ffesom = pf.get_data(resultpath, "CO2f", years, mesh, 
                               how="mean", compute=True, runid=self.runname, silent=True)    

        CO2ffesom = -365.*CO2ffesom/1000. # converting to molC/m2/year
    
    
        labelfesom = 'FESOM air-sea CO$_2$ flux {0}-{1}'.format(self.fyear,self.lyear)
        unitfesom = 'CO$_2$ [mol C m$^{-2}$ yr$^{-1}$]' 
        
        
        # Reading CO2 flux data from Takahashi =========================================

        labeltakashi = 'Takahashi air-sea CO$_2$ flux'
        
        path = self.txtfile
        header = 109                                          # Number of lines in file to skip (with text)
        allvalues = []
        with open(path, 'r') as f:
            for _ in range(header):                             # Skipping text lines
                f.readline()
            for line in f:                                      # Saving all data in matrix "allvalues"
                allvalues.append(line.split())

        allvalues     = np.array(allvalues)                   # Converting list to array
        allvalues     = allvalues.astype(np.float)            # Converting string to float
        lat           = allvalues[:,0]                        # Saving all latitude values in array "lat"
        lat_matrix    = np.unique(lat)                           # 
        lon           = allvalues[:,1]                        # Saving all longitude values in array "lon"
        lon[lon>180.] = lon[lon>180.]-360.                    # Converting lon to values from -180 to 180
        lon_matrix    = np.unique(lon)                           #
        month         = allvalues[:,2]                        # Saving number of month in array "month"      
        flux          = allvalues[:,17]                       # CO2 flux in units [moles C/m2/month] saved in array "flux"

        matrix = np.zeros(shape=(72,40,12))                   # Initializing matrix [lon,lat,month] for gridded sorting of flux values
        for i in range(0,len(flux)):                          # Stepping through all values and adding them to proper position in the matrix
            n = np.where(lat_matrix == lat[i])[0] 
            m = np.where(lon_matrix == lon[i])[0]
            matrix[m,n,int(month[i]-1)] = flux[i]
        matrix[matrix==0] = np.nan                                 # Converting missing values to NaN
        CO2matrix = np.sum(matrix,axis=2)                     # Yearly CO2 flux [moles C/m2/yr]
        CO2matrix = CO2matrix.transpose()                     # Transposing 

        # ==============================================================================
        # Interpolation to fesom's grid

        lonbefore, latbefore = np.meshgrid(lon_matrix, lat_matrix)               # Matrix lon and lat prepared for interpolation

        CO2ftakashi = griddata((lonbefore.ravel(), latbefore.ravel()), CO2matrix.ravel(), (mesh.x2, mesh.y2), method='nearest')
        CO2ftakashi = np.ma.filled(CO2ftakashi, np.nan)

        # apply sea mask to Takashi as in FESOM ----------------------------------------------------------------------------------
        # assumption: there is no ocean where value in FESOM == 0
        CO2ftakashi_ma = np.copy(CO2ftakashi)
        CO2ftakashi_ma[np.isnan(CO2ffesom)] = np.nan
        
        
        # Plotting -------------------------------------------------------------------------------------
        print('\nPlotting CO2 flux \nFESOM min = {0}, max = {1}\nTakashi min = {2}, max = {3}'.format(
                np.nanmin(CO2ffesom),np.nanmax(CO2ffesom),
                np.nanmin(CO2ftakashi_ma),np.nanmax(CO2ftakashi_ma)))

        fig_data = pf.plot(mesh, [CO2ftakashi_ma,CO2ffesom, CO2ftakashi_ma-CO2ffesom], 
                    rowscol= (1,3),
                    levels = (-6,9,31),
                    units=unitfesom, 
                    mapproj=self.mapproj, # robinson projection takes more time!
                    titles=[labeltakashi,labelfesom,
                            'Difference air-sea CO$_2$ flux (model-data)'],
                    figsize = (20,20)
                   )
        # fig export  -------------------------------------------------------------------------------------
        if(self.savefig==True):
            plt.savefig(self.savepath+self.runname+'_'+'CO2flux_Takashi'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                    dpi = 300, bbox_inches='tight')
        plt.show(block=False)  

        # statistics  -------------------------------------------------------------------------------------            
        # preparation of datasets

        # get statistics only from ocean gridpoints (same mask assumption as above)
        #ind_aux = np.where(CO2ffesom != 0)
        #CO2ftakashi_ma_aux = CO2ftakashi_ma[ind_aux]
        
        # get statistics only from valid Takashi points
        ind_stat = np.where(np.isfinite(CO2ftakashi_ma))
        
        if np.isnan(np.min(CO2ftakashi_ma[ind_stat])): print('WARNING: The interpolated Takashi field contains NaNs at depth')
        if np.isnan(np.min(CO2ffesom[ind_stat])): print('WARNING: The FESOM field contains NaNs at depth')

        title = 'Taylor Diagram for CO$_2$ flux'
        plt_Taylor_norm(CO2ftakashi_ma[ind_stat],CO2ffesom[ind_stat],mask=True,title=title)

        # fig export  -------------------------------------------------------------------------------------
        if(self.savefig==True):                
            plt.savefig(self.savepath+self.runname+'_'+'CO2flux_Takashi_Taylor'+'_'+str(years[0])+'to'+str(years[1])+'.png', 
                    dpi = 300, bbox_inches='tight')
        plt.show(block=False) 

In [ ]:
### TESTING ###
if __name__ == "__main__":
    
    # run specification -------------------------------------------------------------------------------------
    runid      =  'fesom'
    resultpath = '/work/ollie/mozeisin/results/f2r1.2/' + 'mo17'
    savepath = '/home/ollie/mozeisin/evaluation/mo_files/'


    meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'


    # period of analysis ------------------------------------------------------------------------------------

    first_year = 1948
    last_year  = 1970

    # Takashi ------------------------------------------------------------------------------------    
    txtfile           = '/work/ollie/projects/MarESys/evaluation/CO2_flux2015_Takahashietal2009_original.txt'
    
    # now test:
    test = CO2f_Takahashi_comp(runid,resultpath,savepath,meshpath,txtfile,first_year,last_year,
                               layerwise=False,savefig=False)